In [1]:
import pandas as pd
import urllib.request, urllib.parse, urllib.error
import json

In [2]:
with open('APIkeys.json') as f:
    keys = json.load(f)
    omdbapi = keys['OMDBapi']

In [3]:
serviceurl = 'http://www.omdbapi.com/?'
apikey = '&apikey='+omdbapi

## Function for printing JSON dataset

In [4]:
def print_json(json_data):
    list_keys=['Title', 'Year', 'Rated', 'Released', 'Runtime', 'Genre', 'Director', 'Writer', 
               'Actors', 'Plot', 'Language', 'Country', 'Awards', 'Ratings', 
               'Metascore', 'imdbRating', 'imdbVotes', 'imdbID']
    print("-"*50)
    for k in list_keys:
        if k in list(json_data.keys()):
            print(f"{k}: {json_data[k]}")
    print("-"*50)

## Function to download poster

In [5]:
def save_poster(json_data):
    import os
    title = json_data['Title']
    poster_url = json_data['Poster']
    # Splits the poster url by '.' and picks up the last string as file extension
    poster_file_extension=poster_url.split('.')[-1]
    # Reads the image file from web
    poster_data = urllib.request.urlopen(poster_url).read()
        
    savelocation=os.getcwd()+'\\'+'Posters'+'\\'
    # Creates new directory if the directory does not exist. Otherwise, just use the existing path.
    if not os.path.isdir(savelocation):
        os.mkdir(savelocation)
    
    filename=savelocation+str(title)+'.'+poster_file_extension
    f=open(filename,'wb')
    f.write(poster_data)
    f.close()

## Function to create/update the local movie database with the data retreived from the web


Saves the movie data (Title, Year, Runtime, Country, Metascore, and IMDB rating) into a local SQLite database called 'movies.sqlite'

In [6]:
#this is the query string
#http://www.omdbapi.com/?t=Toy+Story&y=1995

In [7]:
def save_in_database(json_data):
    
    filename = 'output/IMDB.sqlite'
    
    import sqlite3
    conn = sqlite3.connect(str(filename))
    cur=conn.cursor()
    
    title = json_data['Title']
    # Goes through the json dataset and extracts information if it is available
    if json_data['Year']!='N/A':
        year = json_data['Year']
    else:
        year='NA'
    if json_data['Rated']!='N/A':
        rated = json_data['Rated']
    else:
        rated='NA'
    if json_data['Released']!='N/A':
        released = json_data['Released']
    else:
        released='NA'
    if json_data['Runtime']!='N/A':
        runtime = int(json_data['Runtime'].split()[0])
    else:
        runtime=-1
    if json_data['Awards']!='N/A':
        awards = json_data['Awards']
    else:
        awards='NA'
    if json_data['Metascore']!='N/A':
        metascore = float(json_data['Metascore'])
    else:
        metascore=-1
    if json_data['imdbRating']!='N/A':
        imdb_rating = float(json_data['imdbRating'])
    else:
        imdb_rating=-1
    if json_data['imdbVotes']!='N/A':
        imdb_votes = json_data['imdbVotes']
    else:
        imdb_votes=-1
    if json_data['imdbID']!='N/A':
        imdbId = json_data['imdbID']
    
    
    # SQL commands
    cur.execute('''CREATE TABLE IF NOT EXISTS MovieInfo 
    (Title TEXT, Year TEXT, Rated TEXT, Released TEXT, Runtime INTEGER, Awards TEXT, Metascore REAL, IMDBRating REAL, imdbVotes TEXT, imdbID TEXT)''')
    
    cur.execute('SELECT Title FROM MovieInfo WHERE Title = ? ', (title,))
    row = cur.fetchone()
    
    if row is None:
        cur.execute('''INSERT INTO MovieInfo (Title, Year, Rated, Released, Runtime, Awards, Metascore, IMDBRating, imdbVotes, imdbID )
                VALUES (?,?,?,?,?,?,?,?,?,?)''', (title,year,rated,released,runtime,awards,metascore,imdb_rating,imdb_votes,imdbId))
    else:
        print("Record already found. No update made.")
    
    # Commits the change and close the connection to the database
    conn.commit()
    conn.close()

## Function to print contents of the local database

In [8]:
def print_database(database):
    
    import sqlite3
    conn = sqlite3.connect(str(database))
    cur=conn.cursor()
    
    for row in cur.execute('SELECT * FROM MovieInfo'):
        print(row)
    conn.close()

## Function to save the database content in an CSV file

In [9]:
def save_in_csv(filename, database):
    
    if filename.split('.')[-1]!='csv':
        print ("Filename does not have correct extension. Please try again")
        return None
    
    import pandas as pd
    import sqlite3
    
    conn = sqlite3.connect(str(database))
    #cur=conn.cursor()
    
    df=pd.read_sql_query("SELECT * FROM MovieInfo", conn)
    conn.close()
    
    df.to_csv(filename, index = None, header=True)

## Function to search for information about a movie

In [64]:
def search_movie(title,year):
    if len(title) < 1 or title=='quit': 
        print("Goodbye now...")
        return None

    try:
        url = serviceurl + urllib.parse.urlencode({'t': title})+ '&' + urllib.parse.urlencode({'y': year}) + apikey
        
        print(f'Retrieving the data of "{title}" now... ')
        uh = urllib.request.urlopen(url)
        data = uh.read()
        json_data=json.loads(data)
        
        if json_data['Response']=='True':
            print_json(json_data)
            save_in_database(json_data)
            
        else:
            try:
                first = title.split(' ')[0].lower()
                print(first)
                second = title.split(' ')[1].lower()
                #title = first + second
                url = serviceurl + urllib.parse.urlencode({'t': first})+ '+' + urllib.parse.urlencode({'t': second}) + '&' + urllib.parse.urlencode({'y': year}) + apikey
                print("Error encountered: ",json_data['Error'])
            except IndexError:
                print("Error encountered: ",json_data['Error'])
            
    
    except urllib.error.URLError as e:
        print(f"ERROR: {e.reason}")

## Load Disney titles to get all the information through function search_movie

In [31]:
#Load Disney titles

disney_df = pd.read_csv('output/2021_disney_movies_total_gross.csv')
disney_df['year'] = pd.DatetimeIndex(disney_df['release_date']).year
disney_df.head()

,movie_title,year,release_date,total_gross,inflation_adjusted_gross
0,101 Dalmatians,1961,"Jan 25, 1961",153000000,1386568896
1,101 Dalmatians,1996,"Nov 27, 1996",136189294,235202122
2,102 Dalmatians,2000,"Nov 22, 2000",66941559,105337713
3,1492: Conquest of Paradise,1992,"Oct 9, 1992",7099531,13711759
4,"20,000 Leagues Under the Sea",1954,"Dec 23, 1954",28200000,284065204


In [12]:
# disney_df = pd.read_csv('no_imdb.csv')
# disney_df['year'] = pd.DatetimeIndex(disney_df['release_date']).year
# disney_df.head()

In [19]:
disney_df.count()

movie_title                 618
year                        618
release_date                618
total_gross                 618
inflation_adjusted_gross    618
dtype: int64

## === Test only STARTS here ===

In [14]:
# #test with this df to see if it works as free API call is only 1000 calls per day
# test_df = disney_df.loc[disney_df['movie_title'].isin(['Tron','Good Morning Vietnam'])]
# test_df

In [15]:
# #query for all the movies in our disney list and get the latest information from IMDB

# for i in test_df.index: 
#     title = test_df['movie_title'][i].lower()
#     year = test_df['year'][i]
#     print (test_df['movie_title'][i])
#     print (test_df['year'][i])
#     print ('=====================================')
#     search_movie(title,year)

## === Test only ENDS here ===

In [40]:
#query for all the movies in our disney list and get the latest information from IMDB

for i in disney_df.index: 
    title = disney_df['movie_title'][i].lower()
    year = disney_df['year'][i]
    print (disney_df['movie_title'][i])
    print (disney_df['year'][i])
    print ('=====================================')
    search_movie(title,year)

101 Dalmatians
1961
Retrieving the data of "101 dalmatians" now... 
Error encountered:  Movie not found!
101 Dalmatians
1996
Retrieving the data of "101 dalmatians" now... 
--------------------------------------------------
Title: 101 Dalmatians
Year: 1996
Rated: G
Released: 27 Nov 1996
Runtime: 103 min
Genre: Adventure, Comedy, Crime
Director: Stephen Herek
Writer: Dodie Smith, John Hughes
Actors: Glenn Close, Jeff Daniels, Joely Richardson
Plot: An evil high-fashion designer plots to steal Dalmatian puppies in order to make an extravagant fur coat, but instead creates an extravagant mess.
Language: English, Spanish
Country: United States, United Kingdom
Awards: Nominated for 1 BAFTA Film Award3 wins & 10 nominations total
Ratings: [{'Source': 'Internet Movie Database', 'Value': '5.7/10'}, {'Source': 'Rotten Tomatoes', 'Value': '41%'}, {'Source': 'Metacritic', 'Value': '49/100'}]
Metascore: 49
imdbRating: 5.7
imdbVotes: 109,762
imdbID: tt0115433
---------------------------------------

--------------------------------------------------
Title: A Far Off Place
Year: 1993
Rated: PG
Released: 12 Mar 1993
Runtime: 108 min
Genre: Adventure, Drama, Family
Director: Mikael Salomon
Writer: Lourens van der Post, Robert Caswell, Jonathan Hensleigh
Actors: Reese Witherspoon, Ethan Embry, Jack Thompson
Plot: When a gang of African poachers raid her family's farm, young Nonni, visiting American teenager Harry Winslow and a local bushman guide must escape into the Kalahari desert and reach the nearest town.
Language: English
Country: United States, United Kingdom
Awards: N/A
Ratings: [{'Source': 'Internet Movie Database', 'Value': '6.5/10'}, {'Source': 'Rotten Tomatoes', 'Value': '45%'}]
Metascore: N/A
imdbRating: 6.5
imdbVotes: 3,556
imdbID: tt0106868
--------------------------------------------------
A Goofy Movie
1995
Retrieving the data of "a goofy movie" now... 
--------------------------------------------------
Title: A Goofy Movie
Year: 1995
Rated: G
Released: 07 Apr 1995
Ru

--------------------------------------------------
Title: African Cats
Year: 2011
Rated: G
Released: 22 Apr 2011
Runtime: 89 min
Genre: Documentary, Adventure
Director: Keith Scholey, Alastair Fothergill
Writer: Keith Scholey, John Truby, Owen Newman
Actors: Samuel L. Jackson, Patrick Stewart
Plot: Meet Mara, an endearing lion cub striving to be like her mother; Sita, a cheetah and single mother of five; and Fang, a proud leader of the pride who must defend his family from a rival lion.
Language: English
Country: United States
Awards: 3 nominations
Ratings: [{'Source': 'Internet Movie Database', 'Value': '7.5/10'}, {'Source': 'Rotten Tomatoes', 'Value': '73%'}, {'Source': 'Metacritic', 'Value': '61/100'}]
Metascore: 61
imdbRating: 7.5
imdbVotes: 6,446
imdbID: tt1223236
--------------------------------------------------
Air Bud
1997
Retrieving the data of "air bud" now... 
--------------------------------------------------
Title: Air Bud
Year: 1997
Rated: PG
Released: 01 Aug 1997
Runtim

Error encountered:  Movie not found!
An American Werewolf in Paris
1997
Retrieving the data of "an american werewolf in paris" now... 
--------------------------------------------------
Title: An American Werewolf in Paris
Year: 1997
Rated: R
Released: 25 Dec 1997
Runtime: 98 min
Genre: Comedy, Fantasy, Horror
Director: Anthony Waller
Writer: John Landis, Tim Burns, Tom Stern
Actors: Tom Everett Scott, Julie Delpy, Vince Vieluf
Plot: An American man unwittingly gets involved with French werewolves who have developed a serum allowing them to transform at will.
Language: English, French, German
Country: United Kingdom, Netherlands, Luxembourg, United States, France, Germany
Awards: 3 wins & 1 nomination
Ratings: [{'Source': 'Internet Movie Database', 'Value': '5.1/10'}, {'Source': 'Rotten Tomatoes', 'Value': '7%'}, {'Source': 'Metacritic', 'Value': '31/100'}]
Metascore: 31
imdbRating: 5.1
imdbVotes: 24,800
imdbID: tt0118604
--------------------------------------------------
An Innocent M

--------------------------------------------------
Title: Arachnophobia
Year: 1990
Rated: PG-13
Released: 18 Jul 1990
Runtime: 109 min
Genre: Comedy, Horror, Thriller
Director: Frank Marshall
Writer: Don Jakoby, Al Williams, Wesley Strick
Actors: Jeff Daniels, Julian Sands, John Goodman
Plot: A species of South American killer spider hitches a lift to the U.S. in a coffin and starts to breed and kill.
Language: English, Spanish
Country: United States
Awards: 2 wins & 5 nominations
Ratings: [{'Source': 'Internet Movie Database', 'Value': '6.5/10'}, {'Source': 'Rotten Tomatoes', 'Value': '93%'}, {'Source': 'Metacritic', 'Value': '67/100'}]
Metascore: 67
imdbRating: 6.5
imdbVotes: 68,662
imdbID: tt0099052
--------------------------------------------------
Armageddon
1998
Retrieving the data of "armageddon" now... 
--------------------------------------------------
Title: Armageddon
Year: 1998
Rated: PG-13
Released: 01 Jul 1998
Runtime: 151 min
Genre: Action, Adventure, Sci-Fi
Director: Mi

...

--------------------------------------------------
Title: Confessions of a Teenage Drama Queen
Year: 2004
Rated: PG
Released: 20 Feb 2004
Runtime: 89 min
Genre: Comedy, Family, Music
Director: Sara Sugarman
Writer: Dyan Sheldon, Gail Parent
Actors: Lindsay Lohan, Megan Fox, Adam Garcia
Plot: A teenage girl is convinced that her home city revolves around her until her family packs up and moves to the suburbs, where she finds herself competing for attention.
Language: English
Country: Germany, United States
Awards: 1 win
Ratings: [{'Source': 'Internet Movie Database', 'Value': '4.6/10'}, {'Source': 'Rotten Tomatoes', 'Value': '14%'}, {'Source': 'Metacritic', 'Value': '33/100'}]
Metascore: 33
imdbRating: 4.6
imdbVotes: 28,929
imdbID: tt0361467
--------------------------------------------------
Consenting Adults
1992
Retrieving the data of "consenting adults" now... 
--------------------------------------------------
Title: Consenting Adults
Year: 1992
Rated: R
Released: 16 Oct 1992
Runtim

--------------------------------------------------
Title: D3: The Mighty Ducks
Year: 1996
Rated: PG
Released: 04 Oct 1996
Runtime: 104 min
Genre: Action, Comedy, Drama
Director: Robert Lieberman
Writer: Steven Brill, Kenneth Johnson, Jim Burnstein
Actors: Emilio Estevez, Jeffrey Nordling, David Selby
Plot: The unruly team of hockey misfits grows up a bit and receives scholarships to a prestigious prep school, where they take on the snotty varsity team.
Language: English
Country: United States
Awards: 2 nominations
Ratings: [{'Source': 'Internet Movie Database', 'Value': '5.4/10'}, {'Source': 'Rotten Tomatoes', 'Value': '20%'}]
Metascore: N/A
imdbRating: 5.4
imdbVotes: 26,748
imdbID: tt0116000
--------------------------------------------------
Dan in Real Life
2007
Retrieving the data of "dan in real life" now... 
--------------------------------------------------
Title: Dan in Real Life
Year: 2007
Rated: PG-13
Released: 26 Oct 2007
Runtime: 98 min
Genre: Comedy, Drama, Romance
Director

--------------------------------------------------
Title: Dinosaur
Year: 2000
Rated: PG
Released: 19 May 2000
Runtime: 82 min
Genre: Animation, Adventure, Drama
Director: Eric Leighton, Ralph Zondag
Writer: Thom Enriquez, John Harrison, Robert Nelson Jacobs
Actors: D.B. Sweeney, Julianna Margulies, Samuel E. Wright
Plot: An orphaned dinosaur raised by lemurs joins an arduous trek to a sanctuary after a meteorite shower destroys his family home.
Language: English
Country: United States
Awards: 4 wins & 21 nominations
Ratings: [{'Source': 'Internet Movie Database', 'Value': '6.4/10'}, {'Source': 'Rotten Tomatoes', 'Value': '64%'}, {'Source': 'Metacritic', 'Value': '56/100'}]
Metascore: 56
imdbRating: 6.4
imdbVotes: 59,069
imdbID: tt0130623
--------------------------------------------------
Disney Planes
2013
Retrieving the data of "disney planes" now... 
Error encountered:  Movie not found!
Disney's A Christmas Carol
2009
Retrieving the data of "disney's a christmas carol" now... 
Error 

--------------------------------------------------
Title: Earth
Year: 2009
Rated: N/A
Released: 05 Sep 2009
Runtime: 40 min
Genre: Short, History
Director: Tzu Nyen Ho
Writer: N/A
Actors: Jason Chang, John Low, Erik Wayne Goh
Plot: We see the site of an unknown disaster, the debris of history that constitutes the story of Earth. Upon the site, lay fifty 50 humans oscillating between consciousness and unconsciousness, life and death. Sometimes, one of them emerg
Language: English
Country: Singapore
Awards: N/A
Ratings: [{'Source': 'Internet Movie Database', 'Value': '6.9/10'}]
Metascore: N/A
imdbRating: 6.9
imdbVotes: 10
imdbID: tt1757737
--------------------------------------------------
Ed Wood
1994
Retrieving the data of "ed wood" now... 
--------------------------------------------------
Title: Ed Wood
Year: 1994
Rated: R
Released: 07 Oct 1994
Runtime: 127 min
Genre: Biography, Comedy, Drama
Director: Tim Burton
Writer: Rudolph Grey, Scott Alexander, Larry Karaszewski
Actors: Johnny

IndexError: list index out of range

In [44]:
for i in disney_df.index: 
    title = disney_df['movie_title'][i].lower()
    year = disney_df['year'][i]
    print (title)
    print (year)
    print ('=====================================')

101 dalmatians
dalmatians
1961
101 dalmatians
dalmatians
1996
102 dalmatians
dalmatians
2000
1492: conquest of paradise
conquest
1992
20,000 leagues under the sea
leagues
1954
25th hour
hour
2002
3 men and a baby
men
1987
3 men and a little lady
men
1990
3 ninjas
ninjas
1992
3 ninjas kick back
ninjas
1994
a beautiful planet
beautiful
2016
a bug's life
bug's
1998
a civil action
civil
1998
a far off place
far
1993
a goofy movie
goofy
1995
a gun in betty lou's handbag
gun
1992
a kid in king arthur's court
kid
1995
a lot like love
lot
2005
a low down dirty shame
low
1994
a pyromaniac's love story
pyromaniac's
1995
a simple twist of fate
simple
1994
a stranger among us
stranger
1992
a thousand acres
thousand
1997
a wrinkle in time
wrinkle
2018
adventures in babysitting
in
1987
african cats
cats
2011
air bud
bud
1997
aladdin


IndexError: list index out of range

In [66]:
disney_df.loc[disney_df['movie_title'] =='Endurance']

,movie_title,year,release_date,total_gross,inflation_adjusted_gross
163,Endurance,1999,"May 14, 1999",229128,372669


In [65]:
#Resolving issues with the parameters and starting from where we stopped and continue to the end of list

for i in range(163,len(disney_df)): 
    title = disney_df['movie_title'][i]
    year = disney_df['year'][i]
    print (disney_df['movie_title'][i])
    print (disney_df['year'][i])
    print ('=====================================')
    search_movie(title,year)

Endurance
1999
Retrieving the data of "Endurance" now... 
endurance
Error encountered:  Movie not found!
Enemy of the State
1998
Retrieving the data of "Enemy of the State" now... 
ERROR: Unauthorized
Ernest Goes to Camp
1987
Retrieving the data of "Ernest Goes to Camp" now... 
ERROR: Unauthorized
Ernest Goes to Jail
1990
Retrieving the data of "Ernest Goes to Jail" now... 
ERROR: Unauthorized
Ernest Saves Christmas
1988
Retrieving the data of "Ernest Saves Christmas" now... 
ERROR: Unauthorized
Ernest Scared Stupid
1991
Retrieving the data of "Ernest Scared Stupid" now... 
ERROR: Unauthorized
Eternals
2021
Retrieving the data of "Eternals" now... 
ERROR: Unauthorized
Evita
1996
Retrieving the data of "Evita" now... 
ERROR: Unauthorized
Fantasia
1940
Retrieving the data of "Fantasia" now... 
--------------------------------------------------
Title: Fantasia
Year: 1940
Rated: G
Released: 19 Sep 1941
Runtime: 125 min
Genre: Animation, Family, Fantasy
Director: James Algar, Samuel Armstro

ERROR: Unauthorized
Glory Road
2006
Retrieving the data of "Glory Road" now... 
--------------------------------------------------
Title: Glory Road
Year: 2006
Rated: PG
Released: 13 Jan 2006
Runtime: 118 min
Genre: Biography, Drama, Sport
Director: James Gartner
Writer: Christopher Cleveland, Bettina Gilois
Actors: Josh Lucas, Derek Luke, Austin Nichols
Plot: In 1966, Texas Western coach Don Haskins led the first all-black starting line-up for a college basketball team to the NCAA national championship.
Language: English
Country: United States
Awards: 5 nominations
Ratings: [{'Source': 'Internet Movie Database', 'Value': '7.2/10'}, {'Source': 'Metacritic', 'Value': '58/100'}]
Metascore: 58
imdbRating: 7.2
imdbVotes: 45,390
imdbID: tt0385726
--------------------------------------------------
Record already found. No update made.
Gnomeo and Juliet
2011
Retrieving the data of "Gnomeo and Juliet" now... 
gnomeo
Error encountered:  Movie not found!
Goal! 2: Living the Dream...
2008
Retriev

--------------------------------------------------
Title: Hello Again
Year: 1987
Rated: PG
Released: 06 Nov 1987
Runtime: 96 min
Genre: Comedy, Drama, Fantasy
Director: Frank Perry
Writer: Susan Isaacs
Actors: Shelley Long, Judith Ivey, Gabriel Byrne
Plot: A suburban housewife chokes to death and is brought back to life by a spell cast by her wacky sister.
Language: English
Country: United States
Awards: 1 nomination
Ratings: [{'Source': 'Internet Movie Database', 'Value': '5.3/10'}, {'Source': 'Rotten Tomatoes', 'Value': '10%'}]
Metascore: N/A
imdbRating: 5.3
imdbVotes: 3,450
imdbID: tt0093175
--------------------------------------------------
Record already found. No update made.
Herbie Goes to Monte Carlo
1977
Retrieving the data of "Herbie Goes to Monte Carlo" now... 
ERROR: Unauthorized
Herbie: Fully Loaded
2005
Retrieving the data of "Herbie: Fully Loaded" now... 
ERROR: Unauthorized
Hercules
1997
Retrieving the data of "Hercules" now... 
ERROR: Unauthorized
Hidalgo
2004
Retrievi

ERROR: Unauthorized
Ice Princess
2005
Retrieving the data of "Ice Princess" now... 
ERROR: Unauthorized
In the Army Now
1994
Retrieving the data of "In the Army Now" now... 
ERROR: Unauthorized
Incredibles 2
2018
Retrieving the data of "Incredibles 2" now... 
ERROR: Unauthorized
Indian Summer
1993
Retrieving the data of "Indian Summer" now... 
--------------------------------------------------
Title: Indian Summer
Year: 1993
Rated: PG-13
Released: 23 Apr 1993
Runtime: 97 min
Genre: Comedy, Drama, Romance
Director: Mike Binder
Writer: Mike Binder
Actors: Alan Arkin, Matt Craven, Diane Lane
Plot: Seven friends reunite for a week-long reunion at a summer camp in Ontario they used to attend as children which is now threatened with being closed down.
Language: English
Country: Canada, United States
Awards: N/A
Ratings: [{'Source': 'Internet Movie Database', 'Value': '6.6/10'}, {'Source': 'Rotten Tomatoes', 'Value': '58%'}, {'Source': 'Metacritic', 'Value': '51/100'}]
Metascore: 51
imdbRatin

--------------------------------------------------
Title: Just Visiting
Year: 2001
Rated: PG-13
Released: 06 Apr 2001
Runtime: 88 min
Genre: Adventure, Comedy, Fantasy
Director: Jean-Marie Poiré
Writer: Jean-Marie Poiré, Christian Clavier, John Hughes
Actors: Jean Reno, Christina Applegate, Christian Clavier
Plot: A French count is in England to marry the princess. She's killed. A wizard helps him time travel back before the murder - but ends up in Chicago, April 2000. A descendant helps him. Can he return to the 1100s?
Language: English, French
Country: France, United States
Awards: N/A
Ratings: [{'Source': 'Internet Movie Database', 'Value': '5.7/10'}, {'Source': 'Rotten Tomatoes', 'Value': '33%'}, {'Source': 'Metacritic', 'Value': '38/100'}]
Metascore: 38
imdbRating: 5.7
imdbVotes: 19,098
imdbID: tt0189192
--------------------------------------------------
Record already found. No update made.
Kazaam
1996
Retrieving the data of "Kazaam" now... 
--------------------------------------

...

--------------------------------------------------
Title: Strange Magic
Year: 2015
Rated: PG
Released: 23 Jan 2015
Runtime: 99 min
Genre: Animation, Adventure, Comedy
Director: Gary Rydstrom
Writer: David Berenbaum, Irene Mecchi, Gary Rydstrom
Actors: Evan Rachel Wood, Elijah Kelley, Kristin Chenoweth
Plot: Goblins, elves, fairies and imps, and their misadventures sparked by the battle over a powerful potion.
Language: English
Country: United States
Awards: 1 win & 4 nominations
Ratings: [{'Source': 'Internet Movie Database', 'Value': '5.8/10'}, {'Source': 'Rotten Tomatoes', 'Value': '18%'}, {'Source': 'Metacritic', 'Value': '25/100'}]
Metascore: 25
imdbRating: 5.8
imdbVotes: 10,988
imdbID: tt4191054
--------------------------------------------------
Record already found. No update made.
Summer of Sam
1999
Retrieving the data of "Summer of Sam" now... 
ERROR: Unauthorized
Super Mario Bros.
1993
Retrieving the data of "Super Mario Bros." now... 
ERROR: Unauthorized
Surrogates
2009
Retri

--------------------------------------------------
Title: The Absent Minded Professor
Year: 1961
Rated: Approved
Released: 16 Mar 1961
Runtime: 92 min
Genre: Comedy, Family, Sci-Fi
Director: Robert Stevenson
Writer: Bill Walsh, Samuel W. Taylor
Actors: Fred MacMurray, Nancy Olson, Keenan Wynn
Plot: A college professor invents an anti-gravity substance which a corrupt businessman wants for himself.
Language: English
Country: United States
Awards: Nominated for 3 Oscars. 1 win & 8 nominations total
Ratings: [{'Source': 'Internet Movie Database', 'Value': '6.8/10'}, {'Source': 'Rotten Tomatoes', 'Value': '83%'}, {'Source': 'Metacritic', 'Value': '75/100'}]
Metascore: 75
imdbRating: 6.8
imdbVotes: 8,176
imdbID: tt0054594
--------------------------------------------------
Record already found. No update made.
The Adventures of Huck Finn
1993
Retrieving the data of "The Adventures of Huck Finn" now... 
--------------------------------------------------
Title: The Adventures of Huck Finn
Year

ERROR: Unauthorized
The Country Bears
2002
Retrieving the data of "The Country Bears" now... 
ERROR: Unauthorized
The Crew
2000
Retrieving the data of "The Crew" now... 
ERROR: Unauthorized
The Devil and Max Devlin
1981
Retrieving the data of "The Devil and Max Devlin" now... 
--------------------------------------------------
Title: The Devil and Max Devlin
Year: 1981
Rated: PG
Released: 06 Mar 1981
Runtime: 96 min
Genre: Comedy, Family, Fantasy
Director: Steven Hilliard Stern
Writer: Mary Rodgers, Jimmy Sangster
Actors: Elliott Gould, Bill Cosby, Susan Anspach
Plot: Max, a dead corrupt businessman, makes a Faustian pact with Satan's henchman to drive three people to Hell in exchange for longer life. Soon Max realizes that there still may be good in him.
Language: English
Country: United States
Awards: N/A
Ratings: [{'Source': 'Internet Movie Database', 'Value': '5.1/10'}, {'Source': 'Rotten Tomatoes', 'Value': '18%'}]
Metascore: N/A
imdbRating: 5.1
imdbVotes: 1,289
imdbID: tt0082263


ERROR: Unauthorized
The Last Shot
2004
Retrieving the data of "The Last Shot" now... 
ERROR: Unauthorized
The Last Song
2010
Retrieving the data of "The Last Song" now... 
ERROR: Unauthorized
The Life Aquatic with Steve Zissou
2004
Retrieving the data of "The Life Aquatic with Steve Zissou" now... 
ERROR: Unauthorized
The Light Between Oceans
2016
Retrieving the data of "The Light Between Oceans" now... 
ERROR: Unauthorized
The Lion King
1994
Retrieving the data of "The Lion King" now... 
ERROR: Unauthorized
The Lion King
2019
Retrieving the data of "The Lion King" now... 
ERROR: Unauthorized
The Little Mermaid
1989
Retrieving the data of "The Little Mermaid" now... 
ERROR: Unauthorized
The Lizzie McGuire Movie
2003
Retrieving the data of "The Lizzie McGuire Movie" now... 
ERROR: Unauthorized
The Lone Ranger
2013
Retrieving the data of "The Lone Ranger" now... 
ERROR: Unauthorized
The Lookout
2007
Retrieving the data of "The Lookout" now... 
ERROR: Unauthorized
The Many Adventures of W

ERROR: Unauthorized
Tombstone
1993
Retrieving the data of "Tombstone" now... 
ERROR: Unauthorized
Tomorrowland
2015
Retrieving the data of "Tomorrowland" now... 
ERROR: Unauthorized
Tough Guys
1986
Retrieving the data of "Tough Guys" now... 
ERROR: Unauthorized
Toy Story
1995
Retrieving the data of "Toy Story" now... 
ERROR: Unauthorized
Toy Story 2
1999
Retrieving the data of "Toy Story 2" now... 
ERROR: Unauthorized
Toy Story 3
2010
Retrieving the data of "Toy Story 3" now... 
ERROR: Unauthorized
Toy Story 3D Double Feature
2009
Retrieving the data of "Toy Story 3D Double Feature" now... 
ERROR: Unauthorized
Toy Story 4
2019
Retrieving the data of "Toy Story 4" now... 
ERROR: Unauthorized
Treasure Planet
2002
Retrieving the data of "Treasure Planet" now... 
ERROR: Unauthorized
Trenchcoat
1983
Retrieving the data of "Trenchcoat" now... 
ERROR: Unauthorized
Tron
1982
Retrieving the data of "Tron" now... 
ERROR: Unauthorized
Tron: Legacy
2010
Retrieving the data of "Tron: Legacy" now...

In [ ]:
# #Show downloaded poster of Titanic
# from IPython.display import Image
# Image("Posters/Toy Story.jpg")

## Print the content of the local database

In [68]:
print_database('output/IMDB.sqlite')

('101 Dalmatians', '1996', 'G', '27 Nov 1996', 103, 'Nominated for 1 BAFTA Film Award3 wins & 10 nominations total', 49.0, 5.7, '109,762', 'tt0115433')
('102 Dalmatians', '2000', 'G', '22 Nov 2000', 100, 'Nominated for 1 Oscar. 1 win & 8 nominations total', 35.0, 4.9, '37,045', 'tt0211181')
('1492: Conquest of Paradise', '1992', 'PG-13', '09 Oct 1992', 154, '2 nominations', 47.0, 6.5, '29,898', 'tt0103594')
('20,000 Leagues Under the Sea', '1954', 'G', '20 Jul 1955', 127, 'Won 2 Oscars. 5 wins & 3 nominations total', 83.0, 7.2, '33,019', 'tt0046672')
('25th Hour', '2002', 'R', '10 Jan 2003', 135, '4 wins & 17 nominations', 68.0, 7.6, '176,763', 'tt0307901')
('3 Ninjas', '1992', 'PG', '07 Aug 1992', 84, '1 nomination', 36.0, 5.3, '21,478', 'tt0103596')
('3 Ninjas Kick Back', '1994', 'PG', '06 May 1994', 93, '3 nominations', 39.0, 4.6, '9,141', 'tt0109015')
('A Beautiful Planet', '2016', 'G', '29 Apr 2016', 46, '1 win & 1 nomination', 75.0, 7.3, '1,367', 'tt2800050')
("A Bug's Life", '19

## Save the database content into an Excel file

In [69]:
save_in_csv('output/IMDB.csv','output/IMDB.sqlite')

In [70]:
#check the CSV is exported correctly
df=pd.read_csv('output/IMDB.csv')
df

,Title,Year,Rated,Released,Runtime,Awards,Metascore,IMDBRating,imdbVotes,imdbID
0,101 Dalmatians,1996,G,27 Nov 1996,103,Nominated for 1 BAFTA Film Award3 wins & 10 no...,49.0,5.7,"109,762",tt0115433
1,102 Dalmatians,2000,G,22 Nov 2000,100,Nominated for 1 Oscar. 1 win & 8 nominations t...,35.0,4.9,"37,045",tt0211181
2,1492: Conquest of Paradise,1992,PG-13,09 Oct 1992,154,2 nominations,47.0,6.5,"29,898",tt0103594
3,"20,000 Leagues Under the Sea",1954,G,20 Jul 1955,127,Won 2 Oscars. 5 wins & 3 nominations total,83.0,7.2,"33,019",tt0046672
4,25th Hour,2002,R,10 Jan 2003,135,4 wins & 17 nominations,68.0,7.6,"176,763",tt0307901
...,...,...,...,...,...,...,...,...,...,...
419,The Distinguished Gentleman,1992,R,04 Dec 1992,112,4 wins,45.0,5.9,"15,983",tt0104114
420,The Doctor,1991,PG-13,16 Aug 1991,122,3 nominations,-1.0,7.0,"5,473",tt0101746
421,The Emperor's New Groove,2000,G,15 Dec 2000,78,Nominated for 1 Oscar. 7 wins & 27 nominations...,70.0,7.4,"195,606",tt0120917
422,The Fifth Estate,2013,R,18 Oct 2013,128,2 wins & 3 nominations,49.0,6.2,"40,667",tt1837703


In [ ]:
df.count()

In [73]:
df[df.Title.str.startswith('Star')]

,Title,Year,Rated,Released,Runtime,Awards,Metascore,IMDBRating,imdbVotes,imdbID
375,Star Wars: Episode VII - The Force Awakens,2015,PG-13,18 Dec 2015,138,Nominated for 5 Oscars. 62 wins & 138 nominati...,80.0,7.9,"904,569",tt2488496
376,Star Wars: Episode VIII - The Last Jedi,2017,PG-13,15 Dec 2017,152,Nominated for 4 Oscars. 25 wins & 99 nominatio...,84.0,6.9,"606,579",tt2527336
